### 0. Imports

### 1. Introduction to this notebook

# 2. Transformation

## 2.3 Activities

In [36]:
import pandas as pd
import numpy as np

In [32]:
result_df_parallel_selenium = pd.read_parquet("../data/accommodations/90_day_availability.parquet")

In [33]:
result_df_parallel_selenium

,query_date,activity_date_range_start,activity_date_range_end,activity_name,description,url,image,image2,available_days,available_times,duration,latitude,longitude,price,currency,category,spanish
0,2024-11-02 14:13:58.613494,2024-11-27,2024-12-03,Entrada a La Pedrera,Adentraos en el mundo arquitectónico de Gaudí ...,www.civitatis.com/es/barcelona/entrada-la-pedr...,www.civitatis.com/f/espana/barcelona/entrada-l...,None,"[29, 02, 01, 28, 03, 30, 27]","[[], [], [], [], [], [], []]",1h 30m,41.395187,2.161758,4.00,EUR,Entradas,Español
1,2024-11-02 14:13:58.618562,2024-11-27,2024-12-03,Tour en tuk tuk por Barcelona,¿Os gustaría visitar las maravillas de Gaudí d...,www.civitatis.com/es/barcelona/tour-tuk-tuk-ba...,"www.civitatis.comdata:image/gif;base64,R0lGODl...",www.civitatis.com/f/espana/barcelona/tour-tuk-...,"[27, 03, 30, 28, 02, 29, 01]","[[10:00, 11:00, 12:15, 13:15, 16:00, 16:30, 17...",1 -4h,41.3944849,2.1816996,17.80,EUR,Visitas guiadas y free tours,Español
2,2024-11-02 14:13:58.622561,2024-11-27,2024-12-03,Paseo en catamarán al atardecer con música en ...,Contempla el skyline de Barcelona mientras dis...,www.civitatis.com/es/barcelona/paseo-catamaran...,"www.civitatis.comdata:image/gif;base64,R0lGODl...",www.civitatis.com/f/espana/barcelona/paseo-cat...,"[28, 27, 03, 01, 29, 30, 02]","[[16:30], [16:30], [16:30], [16:30], [16:30], ...",1h 30m,41.37495867288118,2.17849589524371,7.65,EUR,Paseos en barco,Español
3,2024-11-02 14:13:58.625561,2024-11-27,2024-12-03,Teleférico de Montjuïc,Con esta entrada al Teleférico de Montjuïc pod...,www.civitatis.com/es/barcelona/billete-telefer...,"www.civitatis.comdata:image/gif;base64,R0lGODl...",www.civitatis.com/f/espana/barcelona/billete-t...,"[27, 01, 30, 29, 28, 03, 02]","[[10:00], [10:00], [10:00], [10:00], [10:00], ...",None,41.368762,2.163434,3.20,EUR,Entradas,None
4,2024-11-02 14:13:58.630562,2024-11-27,2024-12-03,Free tour por el Parque Güell,En este free tour por el Parque Güell conocere...,www.civitatis.com/es/barcelona/free-tour-parqu...,"www.civitatis.comdata:image/gif;base64,R0lGODl...",www.civitatis.com/f/espana/barcelona/free-tour...,"[02, 29, 28, 03, 27]","[[11:30], [11:30], [11:30], [9:30, 11:30], [11...",1h 30m,41.41508351,2.154768947,2.00,EUR,Visitas guiadas y free tours,Español
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,2024-11-02 14:13:58.778257,2024-12-03,2024-12-09,Tour de los misterios y leyendas de Barcelona,En esta ruta nocturna por Barcelona descubrire...,www.civitatis.com/es/barcelona/tour-misterios-...,"www.civitatis.comdata:image/gif;base64,R0lGODl...",www.civitatis.com/f/espana/barcelona/tour-mist...,None,None,1h 30m,41.387471,2.172001,4.20,EUR,Visitas guiadas y free tours,Español
260,2024-11-02 14:13:58.779258,2024-12-03,2024-12-09,Entrada al Spotify Camp Nou,Si queréis visitar el estadio del Fútbol Club ...,www.civitatis.com/es/barcelona/camp-nou-experi...,"www.civitatis.comdata:image/gif;base64,R0lGODl...",www.civitatis.com/f/espana/barcelona/camp-nou-...,None,None,1h 30m,41.380053,2.121457,2.91,EUR,Entradas,Español
261,2024-11-02 14:13:58.780262,2024-12-03,2024-12-09,Free tour de los misterios y leyendas del Barr...,Acompañadnos en este free tour por Barcelona e...,www.civitatis.com/es/barcelona/free-tour-miste...,"www.civitatis.comdata:image/gif;base64,R0lGODl...",www.civitatis.com/f/espana/barcelona/free-tour...,None,None,2 horas,41.3838057,2.1757634,2.00,EUR,Visitas guiadas y free tours,Español
262,2024-11-02 14:13:58.781265,2024-12-03,2024-12-09,"Excursión a Gerona, Figueras y Museo Dalí",En esta excursión a Gerona y Figueras conocere...,www.civitatis.com/es/barcelona/excursion-museo...,"www.civitatis.comdata:image/gif;base64,R0lGODl...",www.civitatis.com/f/espana/barcelona/excursion...,None,None,9h - 10h 45m,41.3940811,2.1826233,23.70,EUR,Excursiones de un día,Español


### 2.3.1 Cleaning neccessary

1. Remove duplicates
2. Keep only the good image: The second if it has 2, otherwise the first.
3. Take unique, non null and non zero latitudes and longitudes, to convert to address.
4. Format the available dates and times

Careful as some non-available times might mean they do not apply, i.e. tickets.

##### 2.3.1.1 Remove duplicates

By activity_name and activity_date_range_start

In [34]:
result_df_parallel_selenium[result_df_parallel_selenium.duplicated(subset=["activity_date_range_start","activity_name"],keep=False)].sort_values(by="activity_name").head()

,query_date,activity_date_range_start,activity_date_range_end,activity_name,description,url,image,image2,available_days,available_times,duration,latitude,longitude,price,currency,category,spanish
226,2024-11-02 14:13:58.688766,2024-12-03,2024-12-09,Barcelona Card Express,Con la tarjeta Barcelona Card Express obtendrá...,www.civitatis.com/es/barcelona/barcelona-card-...,www.civitatis.com/f/espana/barcelona/barcelona...,None,"[04, 06, 09, 03, 05, 08, 07]","[[0:00], [0:00], [0:00], [0:00], [0:00], [0:00...",2 días,0,0,5.40,EUR,Entradas,None
235,2024-11-02 14:13:58.724155,2024-12-03,2024-12-09,Barcelona Card Express,Con la tarjeta Barcelona Card Express obtendrá...,www.civitatis.com/es/barcelona/barcelona-card-...,www.civitatis.com/f/espana/barcelona/barcelona...,None,"[04, 06, 07, 05, 03, 08, 09]","[[0:00], [0:00], [0:00], [0:00], [0:00], [0:00...",2 días,0,0,5.40,EUR,Entradas,None
234,2024-11-02 14:13:58.706054,2024-12-03,2024-12-09,Espectáculo flamenco en el Tablao El Duende,Asistiendo a un espectáculo flamenco en el Tab...,www.civitatis.com/es/barcelona/espectaculo-fla...,"www.civitatis.comdata:image/gif;base64,R0lGODl...",www.civitatis.com/f/espana/barcelona/espectacu...,"[07, 08, 09, 06, 04, 05, 03]","[[19:00, 20:15], [19:00, 20:15], [19:00, 20:15...",50 minutos,41.3789078,2.1747321,6.75,EUR,Espectáculos,None
243,2024-11-02 14:13:58.738170,2024-12-03,2024-12-09,Espectáculo flamenco en el Tablao El Duende,Asistiendo a un espectáculo flamenco en el Tab...,www.civitatis.com/es/barcelona/espectaculo-fla...,"www.civitatis.comdata:image/gif;base64,R0lGODl...",www.civitatis.com/f/espana/barcelona/espectacu...,"[07, 04, 06, 09, 03, 05, 08]","[[19:00, 20:15], [19:00, 20:15], [19:00, 20:15...",50 minutos,41.3789078,2.1747321,6.75,EUR,Espectáculos,None
237,2024-11-02 14:13:58.730169,2024-12-03,2024-12-09,Espectáculo flamenco en el Tablao de Carmen,¿Queréis experimentar lo mejor de la cultura e...,www.civitatis.com/es/barcelona/espectaculo-fla...,"www.civitatis.comdata:image/gif;base64,R0lGODl...",www.civitatis.com/f/espana/barcelona/espectacu...,"[05, 09, 04, 07, 03]","[[], [], [], [], []]",1 hora,41.3689218,2.1488681,17.00,EUR,Espectáculos,None


In [35]:
result_df_parallel_selenium.drop_duplicates(subset=["activity_date_range_start","activity_name"], inplace=True)

#### 2.3.1.2 Keep only the good image: The second if it has 2, otherwise the first.

In [39]:
result_df_parallel_selenium["image"] = np.where(~result_df_parallel_selenium["image2"].isna(), result_df_parallel_selenium["image2"], result_df_parallel_selenium["image"])
result_df_parallel_selenium.drop(columns="image2",inplace=True)

In [40]:
result_df_parallel_selenium.head()

,query_date,activity_date_range_start,activity_date_range_end,activity_name,description,url,image,available_days,available_times,duration,latitude,longitude,price,currency,category,spanish
0,2024-11-02 14:13:58.613494,2024-11-27,2024-12-03,Entrada a La Pedrera,Adentraos en el mundo arquitectónico de Gaudí ...,www.civitatis.com/es/barcelona/entrada-la-pedr...,www.civitatis.com/f/espana/barcelona/entrada-l...,"[29, 02, 01, 28, 03, 30, 27]","[[], [], [], [], [], [], []]",1h 30m,41.395187,2.161758,4.00,EUR,Entradas,Español
1,2024-11-02 14:13:58.618562,2024-11-27,2024-12-03,Tour en tuk tuk por Barcelona,¿Os gustaría visitar las maravillas de Gaudí d...,www.civitatis.com/es/barcelona/tour-tuk-tuk-ba...,www.civitatis.com/f/espana/barcelona/tour-tuk-...,"[27, 03, 30, 28, 02, 29, 01]","[[10:00, 11:00, 12:15, 13:15, 16:00, 16:30, 17...",1 -4h,41.3944849,2.1816996,17.80,EUR,Visitas guiadas y free tours,Español
2,2024-11-02 14:13:58.622561,2024-11-27,2024-12-03,Paseo en catamarán al atardecer con música en ...,Contempla el skyline de Barcelona mientras dis...,www.civitatis.com/es/barcelona/paseo-catamaran...,www.civitatis.com/f/espana/barcelona/paseo-cat...,"[28, 27, 03, 01, 29, 30, 02]","[[16:30], [16:30], [16:30], [16:30], [16:30], ...",1h 30m,41.37495867288118,2.17849589524371,7.65,EUR,Paseos en barco,Español
3,2024-11-02 14:13:58.625561,2024-11-27,2024-12-03,Teleférico de Montjuïc,Con esta entrada al Teleférico de Montjuïc pod...,www.civitatis.com/es/barcelona/billete-telefer...,www.civitatis.com/f/espana/barcelona/billete-t...,"[27, 01, 30, 29, 28, 03, 02]","[[10:00], [10:00], [10:00], [10:00], [10:00], ...",None,41.368762,2.163434,3.20,EUR,Entradas,None
4,2024-11-02 14:13:58.630562,2024-11-27,2024-12-03,Free tour por el Parque Güell,En este free tour por el Parque Güell conocere...,www.civitatis.com/es/barcelona/free-tour-parqu...,www.civitatis.com/f/espana/barcelona/free-tour...,"[02, 29, 28, 03, 27]","[[11:30], [11:30], [11:30], [9:30, 11:30], [11...",1h 30m,41.41508351,2.154768947,2.00,EUR,Visitas guiadas y free tours,Español


#### 2.3.1.3 Take unique, non null and non zero latitudes and longitudes, to convert to address.

In [ ]:
from dotenv import load_dotenv
import os 
load_dotenv()

True

In [6]:
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

In [ ]:
import aiohttp
import asyncio

async def get_address(lat: float, lon: float, api_key: str) -> str:
    url = f"https://maps.googleapis.com/maps/api/geocode/json?latlng={lat},{lon}&key={api_key}"
    
    async with aiohttp.ClientSession() as session:
        async with session.get(url) as response:
            data = await response.json()
            
            if data.get("status") == "OK":
                address = data["results"][0]["formatted_address"]
                return address
            else:
                return "Address not found"

# Example usage
async def main():
    api_key = GOOGLE_API_KEY
    lat, lon = 40.714224, -73.961452
    address = await get_address(lat, lon, api_key)
    print("Address:", address)


In [48]:
non_zero = ~(result_df_parallel_selenium[["latitude", "longitude"]] == 0).all(axis=1)
not_nan = result_df_parallel_selenium[["latitude", "longitude"]].notna().all(axis=1)
unique_combinations = result_df_parallel_selenium.loc[non_zero & not_nan,["latitude","longitude"]].drop_duplicates()

In [ ]:
unique_combinations

#### 2.3.1.4 Format the available dates and times

1. Add address to location

In [8]:
import aiohttp
import asyncio

async def get_address(lat: float, lon: float, api_key: str) -> str:
    url = f"https://maps.googleapis.com/maps/api/geocode/json?latlng={lat},{lon}&key={api_key}"
    
    async with aiohttp.ClientSession() as session:
        async with session.get(url) as response:
            data = await response.json()
            
            if data.get("status") == "OK":
                address = data["results"][0]["formatted_address"]
                return address
            else:
                return "Address not found"

# Example usage
async def main():
    api_key = GOOGLE_API_KEY
    lat, lon = 40.714224, -73.961452
    address = await get_address(lat, lon, api_key)
    print("Address:", address)



In [9]:
await main()

Address: 277 Bedford Ave, Brooklyn, NY 11211, USA
